In [22]:
import pandas as pd
import re
import pickle
from time import sleep
from tqdm import tqdm_notebook as tqdm


In [13]:
df = pd.read_csv("data/wikihowSep.csv")
df = df.dropna()
tasks = df.title.unique().tolist()
# remove number from tasks
tasks = list(set([re.sub(r'\W+', ' ', i[:-1].lower()) if str.isnumeric(i[-1]) else re.sub(r'\W+', ' ', i.lower()) for i in tasks]))


In [14]:
import http.client, urllib.request, urllib.parse, urllib.error, base64
import json

def getVectors(queries):
    headers = {
        # Request headers
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': '3d886bf041574645a097aab80bc26f0e',
    }

    params = urllib.parse.urlencode({})
    
    try:
        conn = http.client.HTTPSConnection('api.msturing.org')
#         conn.request("POST", "/gen/encode?%s" % params, '{"queries": ["how to make gingerbread people (in grams)", "test AI"]}', headers)
        conn.request("POST", "/gen/encode?%s" % params, str({"queries": queries}).replace("\'", "\""), headers)
        response = conn.getresponse()
        data = response.read()
        data = json.loads(data)
        conn.close()
    except Exception as e:
        print(data)
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
    return {data[i]['query']:data[i]['vector'] for i in range(len(data))}

In [23]:
import glob
files = []
for file in glob.glob("wikihow_2_ms_encoded_vector/*"):
    files.append(file)

ranges = [(i, i+50) for i in range(0, len(tasks), 50)]

finish = []
for f in files:
    try:
        name = f.split("/")[1].split("-")
        finish.append((int(name[0]), int(name[1])))
    except:
        continue
todo = list(set.difference(set(ranges), set(finish)))        

In [24]:
len(todo)

932

In [25]:
for i in tqdm(todo[::-1], total=len(todo)):
    try:
        data = getVectors(tasks[i[0]:i[1]])
        with open("wikihow_2_ms_encoded_vector/%s-%s" % (str(i[0]), str(i[1])), 'wb') as f:
            pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
    except:
        sleep(60)

b'All 1 requests failed. Invoking turing.agiencoderv4_02082019:131992289164512738 on DS1PPF8BC4F6E2D failed due to timeout'
b'All 1 requests failed. Shared memory request 637085160902728825 to turing.agiencoderv4_02082019:131992289164512738 instance 2 timed out'
b'All 1 requests failed. Shared memory request 637084800854294903 to turing.agiencoderv4_02082019:131992289164512738 instance 2 timed out'
b'All 1 requests failed. Invoking turing.agiencoderv4_02082019:131992289164512738 on DS1PPF8BC4F6E2D failed due to timeout'
b'All 1 requests failed. Invoking turing.agiencoderv4_02082019:131992289164512738 on DS2PPF062D4B1F3 failed due to timeout'
b'All 1 requests failed. Invoking turing.agiencoderv4_02082019:131992289164512738 on DS1PPF09FA63DA6 failed due to timeout'
b'All 1 requests failed. Invoking turing.agiencoderv4_02082019:131992289164512738 on DS1PPF8BC4F6E2D failed due to timeout'
b'All 1 requests failed. Invoking turing.agiencoderv4_02082019:131992289164512738 on DS1PPF09FA63DA6 f

KeyboardInterrupt: 

In [234]:
def load_dict(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [238]:
tmp1 = load_dict("wikihow_2_ms_encoded_vector/0-50")
tmp2 = load_dict("wikihow_2_ms_encoded_vector/50-100")